In [51]:
import pandas as pd
import locale
import glob
import chardet

# Load data

In [52]:
def read_data_per_file(file: str) -> pd.DataFrame:
    """
    Reads data from a CSV file and returns it as a pandas DataFrame. Handles skipping rows

    Args:
        file (str): The path to the CSV file.

    Returns:
        pd.DataFrame: The DataFrame containing the data from the CSV file.
    """
    # Detect the encoding
    with open(file, 'rb') as f:
        result = chardet.detect(f.read())
        encoding = result['encoding']
    data =  pd.read_csv(file, encoding=encoding, skiprows=2)
        
    return data

In [53]:
def load_visitor_counts_data(
    data_folder: str,
) -> pd.DataFrame:
    """
    Loads visitor counts data from multiple CSV files in the specified folder and deletes some unnecessary column.

    Args:
        data_folder (str): The path to the folder containing the CSV files.

    Returns:
        pd.DataFrame: The DataFrame containing the visitor counts data.
    """
    # Read data from CSV files

    raw_visitor_counts = pd.concat([read_data_per_file(
        file
    ) for file in glob.glob(f"{data_folder}/*.csv")])
    
    # Drop last empty column
    visitor_counts = raw_visitor_counts.drop(columns=["Unnamed: 96"])

    return visitor_counts

visitor_counts = load_visitor_counts_data(data_folder="data\manual_visitor_counts")

# Preprocess data

In [54]:
def parse_german_dates(
    df: pd.DataFrame,
    date_column_name: str
) -> pd.DataFrame:
    """
    Parses German dates in the specified date column of the DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame containing the date column.
        date_column_name (str): The name of the date column.

    Returns:
        pd.DataFrame: The DataFrame with parsed German dates.
    """
    
    # Set German locale
    locale.setlocale(locale.LC_TIME, "de_DE.UTF-8")
    
    # Mapping of German month names to their English equivalents
    month_map = {
        "Jan.": "Jan",
        "Feb.": "Feb",
        "März": "Mar",
        "Apr.": "Apr",
        "Mai": "May",
        "Juni": "Jun",
        "Juli": "Jul",
        "Aug.": "Aug",
        "Sep.": "Sep",
        "Okt.": "Oct",
        "Nov.": "Nov",
        "Dez.": "Dec"
    }

    # Replace German month names with English equivalents
    for german, english in month_map.items():
        df[date_column_name] = df[date_column_name].str.replace(german, english)

    # Parse the dates
    df[date_column_name] = pd.to_datetime(df[date_column_name], format="mixed")

    return df

visitor_counts_parsed_dates = parse_german_dates(df=visitor_counts, date_column_name="Time")

In [55]:
#for some reason there are duplicated index

visitor_counts_parsed_dates.index.duplicated()

array([False, False, False, ...,  True,  True,  True])

In [56]:
# reset the index to avoid future problems
visitor_counts_parsed_dates = visitor_counts_parsed_dates.reset_index(drop=True)

After reviewing all the columns to check for inconsistencies I arrived to a list of tasks that should be done to clean de dataset:

- Drop repeated columns listed in "to_drop"
- Create "Bucina_Multi IN" as the sum of cyclist and pedestrians it's not there
- Rename columns that had wrong names
- Transform to NaN every value in selected columns before "2021-06-18 00:00:00" as the sensor was not installed.

In [57]:
#lists an dictionaries to drop and rename

to_drop = ['Brechhäuslau Fußgänger IN', 'Brechhäuslau Fußgänger OUT', 'Waldhausreibe Channel 1 IN', 'Waldhausreibe Channel 2 OUT']

to_create = ['Bucina_Multi IN']


to_rename = {'Bucina IN': 'Bucina PYRO IN',
          'Bucina OUT': 'Bucina PYRO OUT',
          'Gsenget IN.1': 'Gsenget Fußgänger IN',
          'Gsenget OUT.1': 'Gsenget Fußgänger OUT',
          'Gfäll Fußgänger IN' : 'Gfäll IN',
          'Gfäll Fußgänger OUT': 'Gfäll OUT',
          'Fredenbrücke Fußgänger IN' : 'Fredenbrücke IN',
          'Fredenbrücke Fußgänger OUT': 'Fredenbrücke OUT',
          'Diensthüttenstraße Fußgänger IN': 'Diensthüttenstraße IN' ,
          'Diensthüttenstraße Fußgänger OUT': 'Diensthüttenstraße OUT',
          'Racheldiensthütte Cyclist OUT' : 'Racheldiensthütte Fahrräder OUT',
          'Racheldiensthütte Pedestrian IN' : 'Racheldiensthütte Fußgänger IN',
          'Racheldiensthütte Pedestrian OUT' : 'Racheldiensthütte Fußgänger OUT',
          'Sagwassersäge Fußgänger IN' : 'Sagwassersäge IN',
          'Sagwassersäge Fußgänger OUT': 'Sagwassersäge OUT',
          'Schwarzbachbrücke Fußgänger IN' : 'Schwarzbachbrücke IN',
          'Schwarzbachbrücke Fußgänger OUT' : 'Schwarzbachbrücke OUT',
          'NPZ_Falkenstein IN' : 'Falkenstein 1 PYRO IN',
          'NPZ_Falkenstein OUT' : 'Falkenstein 1 PYRO OUT',
          'TFG_Falkenstein_1 Fußgänger zum Parkplatz' : 'Falkenstein 1 OUT',
          'TFG_Falkenstein_1 Fußgänger zum HZW' : 'Falkenstein 1 IN',
          'TFG_Falkenstein_2 Fußgänger In Richtung Parkplatz' : 'Falkenstein 2 OUT',
          'TFG_Falkenstein_2 Fußgänger In Richtung TFG' : 'Falkenstein 2 IN',
          'TFG_Lusen_1 IN' : 'Lusen 1 PYRO IN',
          'TFG_Lusen_1 OUT' : 'Lusen 1 PYRO OUT',
          'TFG_Lusen_1 Fußgänger Richtung TFG': 'Lusen 1 EVO IN',
          'TFG_Lusen_1 Fußgänger Richtung Parkplatz' : 'Lusen 1 EVO OUT',
          'TFG_Lusen_2 Fußgänger Richtung Vögel am Waldrand': 'Lusen 2 IN',
          'TFG_Lusen_2 Fußgänger Richtung Parkplatz' : 'Lusen 2 OUT',
          'TFG_Lusen_3 TFG Lusen 3 IN': 'Lusen 3 IN',
          'TFG_Lusen_3 TFG Lusen 3 OUT': 'Lusen 3 OUT',
          'Waldspielgelände_1 IN': 'Waldspielgelände IN',
          'Waldspielgelände_1 OUT': 'Waldspielgelände OUT',
          'Wistlberg Fußgänger IN' : 'Wistlberg IN',
          'Wistlberg Fußgänger OUT' : 'Wistlberg OUT'
          }

to_nan = ['Trinkwassertalsperre_MULTI Fußgänger IN',
          'Trinkwassertalsperre_MULTI Fußgänger OUT',
          'Trinkwassertalsperre_MULTI Fahrräder IN',
          'Trinkwassertalsperre_MULTI Fahrräder OUT']

In [58]:
#Create copy of visitors_counts_parsed_dates
df = visitor_counts_parsed_dates.copy()

# Rename columns
df.rename(columns=to_rename, inplace=True)

# Drop columns
df.drop(columns=to_drop, inplace=True)

# Create Bucina Multi IN
df['Bucina_Multi IN'] = df["Bucina_Multi Fahrräder IN"] + df["Bucina_Multi Fußgänger IN"]

### Before assigning Nan values to a certain period there are problems in our datetime variable to solve

In [59]:
# Calculate the difference between consecutive dates
intervals = df.Time.diff().dropna()

intervals.value_counts()

Time
0 days 01:00:00         61328
0 days 02:00:00             7
0 days 00:00:00             7
-2556 days +01:00:00        1
Name: count, dtype: int64

In [60]:
# Find the first interval to use as a reference
reference_interval = intervals.iloc[0]

# Check where the intervals differ from the reference interval
different_intervals = intervals[intervals != reference_interval]

print("Intervals that differ from the reference interval:")
print(different_intervals)

Intervals that differ from the reference interval:
1994         0 days 02:00:00
1995         0 days 00:00:00
10898        0 days 02:00:00
10899        0 days 00:00:00
19634        0 days 02:00:00
19635        0 days 00:00:00
28370        0 days 02:00:00
28371        0 days 00:00:00
37106        0 days 02:00:00
37107        0 days 00:00:00
45842        0 days 02:00:00
45843        0 days 00:00:00
52584   -2556 days +01:00:00
54602        0 days 02:00:00
54603        0 days 00:00:00
Name: Time, dtype: timedelta64[ns]


While checking the rows we discovered that problematic rows are all NaN

In [61]:
pd.set_option('display.max_columns', None)
df[df.loc[:, df.columns != "Time"].isna().all(axis=1)]

,Time,Bayerisch Eisenstein IN,Bayerisch Eisenstein OUT,Bayerisch Eisenstein Fußgänger IN,Bayerisch Eisenstein Fußgänger OUT,Bayerisch Eisenstein Fahrräder IN,Bayerisch Eisenstein Fahrräder OUT,Brechhäuslau IN,Brechhäuslau OUT,Bucina PYRO IN,Bucina PYRO OUT,Bucina_Multi OUT,Bucina_Multi Fußgänger IN,Bucina_Multi Fahrräder IN,Bucina_Multi Fahrräder OUT,Bucina_Multi Fußgänger OUT,Deffernik IN,Deffernik OUT,Deffernik Fahrräder IN,Deffernik Fahrräder OUT,Deffernik Fußgänger IN,Deffernik Fußgänger OUT,Diensthüttenstraße IN,Diensthüttenstraße OUT,Felswandergebiet IN,Felswandergebiet OUT,Ferdinandsthal IN,Ferdinandsthal OUT,Fredenbrücke IN,Fredenbrücke OUT,Gfäll IN,Gfäll OUT,Gsenget IN,Gsenget OUT,Gsenget Fußgänger IN,Gsenget Fußgänger OUT,Gsenget Fahrräder IN,Gsenget Fahrräder OUT,Klingenbrunner Wald IN,Klingenbrunner Wald OUT,Klingenbrunner Wald Fußgänger IN,Klingenbrunner Wald Fußgänger OUT,Klingenbrunner Wald Fahrräder IN,Klingenbrunner Wald Fahrräder OUT,Klosterfilz IN,Klosterfilz OUT,Klosterfilz Fußgänger IN,Klosterfilz Fußgänger OUT,Klosterfilz Fahrräder IN,Klosterfilz Fahrräder OUT,Falkenstein 1 PYRO IN,Falkenstein 1 PYRO OUT,Racheldiensthütte IN,Racheldiensthütte OUT,Racheldiensthütte Fahrräder IN,Racheldiensthütte Fahrräder OUT,Racheldiensthütte Fußgänger IN,Racheldiensthütte Fußgänger OUT,Sagwassersäge IN,Sagwassersäge OUT,Scheuereck IN,Scheuereck OUT,Schillerstraße IN,Schillerstraße OUT,Schwarzbachbrücke IN,Schwarzbachbrücke OUT,Falkenstein 1 OUT,Falkenstein 1 IN,Falkenstein 2 OUT,Falkenstein 2 IN,TFG_Lusen IN,TFG_Lusen OUT,Lusen 1 EVO IN,Lusen 1 EVO OUT,Lusen 2 IN,Lusen 2 OUT,Lusen 3 IN,Lusen 3 OUT,Trinkwassertalsperre IN,Trinkwassertalsperre OUT,Trinkwassertalsperre_MULTI IN,Trinkwassertalsperre_MULTI OUT,Trinkwassertalsperre_MULTI Fußgänger IN,Trinkwassertalsperre_MULTI Fußgänger OUT,Trinkwassertalsperre_MULTI Fahrräder IN,Trinkwassertalsperre_MULTI Fahrräder OUT,Waldhausreibe IN,Waldhausreibe OUT,Waldspielgelände IN,Waldspielgelände OUT,Wistlberg IN,Wistlberg OUT,Bucina_Multi IN
1994,2018-03-25 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10898,2019-03-31 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19634,2020-03-29 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28370,2021-03-28 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37106,2022-03-27 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45842,202

In [62]:
print("Example of problem: At the end of march every year skips a line")
print(df[["Time", "Bayerisch Eisenstein IN"]][1992: ].head(5))

Example of problem: At the end of march every year skips a line
                    Time  Bayerisch Eisenstein IN
1992 2018-03-25 00:00:00                      NaN
1993 2018-03-25 01:00:00                      NaN
1994 2018-03-25 03:00:00                      NaN
1995 2018-03-25 03:00:00                      NaN
1996 2018-03-25 04:00:00                      NaN


We decided to change every repeated row to fill the "02:00" missing, and imputate that row with the values from the inmediately preceading row

In [63]:
#identify wrong indexes

index_wrong_time = df[df.loc[:, df.columns != "Time"].isna().all(axis=1)].index

# sustract one hour
df.loc[index_wrong_time, 'Time'] = df.loc[index_wrong_time, 'Time'] - pd.Timedelta(hours=1)


In [64]:
# imputate values from the preceding row,

for idx in index_wrong_time:
    if idx > 0:  # Ensure the preceding row exists\n",
        # Copy values from the preceding row\n",
        df.loc[idx, df.columns != 'Time'] = df.loc[idx - 1, df.columns != 'Time']

Now for further work we can set the Time column as index

In [65]:
df = df.set_index('Time').sort_index()

Transform to NaN every value in selected columns before \"2021-06-18 00:00:00\" as the sensor was not installed

In [66]:
print("Columns to edit: ", to_nan)

Columns to edit:  ['Trinkwassertalsperre_MULTI Fußgänger IN', 'Trinkwassertalsperre_MULTI Fußgänger OUT', 'Trinkwassertalsperre_MULTI Fahrräder IN', 'Trinkwassertalsperre_MULTI Fahrräder OUT']


To test models with a simplified but coherent df, this should be done:

- Create a column for the sum of IN and OUT, to have a general representation of traffic with no direction.\n",

- From now on there are two possible ways: we either work with recent data (post replacement of sensors) or merge data for a common position, this means that when a column such as Bucina_IN goes NaN you complete that column with Bucina_Multi_IN."

In [67]:
# Create a mapping to match the names on  visitor_counting_devices.shp (still needs work)

mapping = {'Bayerisch Eisenstein IN': 'Bayerisch Eisenstein',
'Bayerisch Eisenstein OUT': 'Bayerisch Eisenstein',
'Bayerisch Eisenstein Fußgänger IN': 'Bayerisch Eisenstein',
'Bayerisch Eisenstein Fußgänger OUT': 'Bayerisch Eisenstein',
'Bayerisch Eisenstein Fahrräder IN': 'Bayerisch Eisenstein',
'Bayerisch Eisenstein Fahrräder OUT': 'Bayerisch Eisenstein', 
'Brechhäuslau IN': 'Brechhäuslau',
'Brechhäuslau OUT': 'Brechhäuslau', 
'Brechhäuslau Fußgänger IN': 'Brechhäuslau',
'Brechhäuslau Fußgänger OUT': 'Brechhäuslau', 
'Bucina IN': 'Bučina', 
'Bucina OUT': 'Bučina',
'Bucina_Multi OUT': 'Bučina', 
'Bucina_Multi Fußgänger IN': 'Bučina',
'Bucina_Multi Fahrräder IN': 'Bučina', 
'Bucina_Multi Fahrräder OUT': 'Bučina',
'Bucina_Multi Fußgänger OUT': 'Bučina', 
'Deffernik IN': 'Deffernik', 
'Deffernik OUT': 'Deffernik', 
'Deffernik Fahrräder IN': 'Deffernik',  
'Deffernik Fahrräder OUT': 'Deffernik', 
'Deffernik Fußgänger IN': 'Deffernik',  
'Deffernik Fußgänger OUT': 'Deffernik', 
'Diensthüttenstraße Fußgänger IN': 'Diensthüttenstraße',  
'Diensthüttenstraße Fußgänger OUT': 'Diensthüttenstraße', 
'Felswandergebiet IN': 'Felswandergebiet',  
'Felswandergebiet OUT': 'Felswandergebiet', 
'Ferdinandsthal IN': 'Ferdinandsthal',
'Ferdinandsthal OUT': 'Ferdinandsthal', 
'Fredenbrücke Fußgänger IN': 'Fredenbrücke',
'Fredenbrücke Fußgänger OUT': 'Fredenbrücke',
'Gfäll Fußgänger IN': 'Gfäll',
'Gfäll Fußgänger OUT': 'Gfäll', 
'Gsenget IN': 'Gsenget', 
'Gsenget OUT': 'Gsenget', 
'Gsenget IN.1': 'Gsenget',
'Gsenget OUT.1': 'Gsenget', 
'Gsenget Fahrräder IN': 'Gsenget', 
'Gsenget Fahrräder OUT': 'Gsenget',
'Klingenbrunner Wald IN': 'Klingenbrunner Wald',
'Klingenbrunner Wald OUT': 'Klingenbrunner Wald',
'Klingenbrunner Wald Fußgänger IN': 'Klingenbrunner Wald', 
'Klingenbrunner Wald Fußgänger OUT': 'Klingenbrunner Wald',
'Klingenbrunner Wald Fahrräder IN': 'Klingenbrunner Wald', 
'Klingenbrunner Wald Fahrräder OUT': 'Klingenbrunner Wald',
'Klosterfilz IN': 'Klosterfilz',
'Klosterfilz OUT': 'Klosterfilz', 
'Klosterfilz Fußgänger IN': 'Klosterfilz',
'Klosterfilz Fußgänger OUT': 'Klosterfilz', 
'Klosterfilz Fahrräder IN': 'Klosterfilz',
'Klosterfilz Fahrräder OUT': 'Klosterfilz', 
'NPZ_Falkenstein IN': 'TFG Falkenstein 1',
'NPZ_Falkenstein OUT': 'TFG Falkenstein 1', 
'Racheldiensthütte IN': 'Racheldiensthütte',
'Racheldiensthütte OUT': 'Racheldiensthütte',
'Racheldiensthütte Fahrräder IN': 'Racheldiensthütte', 
'Racheldiensthütte Cyclist OUT': 'Racheldiensthütte',
'Racheldiensthütte Pedestrian IN': 'Racheldiensthütte', 
'Racheldiensthütte Pedestrian OUT': 'Racheldiensthütte',
'Sagwassersäge Fußgänger IN': 'Sagwassersäge', 
'Sagwassersäge Fußgänger OUT': 'Sagwassersäge',
'Scheuereck IN': 'Scheuereck',
'Scheuereck OUT': 'Scheuereck', 
'Schillerstraße IN': 'Schillerstraße',
'Schillerstraße OUT': 'Schillerstraße', 
'Schwarzbachbrücke Fußgänger IN': 'Schwarzbachbrücke',
'Schwarzbachbrücke Fußgänger OUT': 'Schwarzbachbrücke',
'TFG_Falkenstein_1 Fußgänger zum Parkplatz': 'TFG Falkenstein 1',
'TFG_Falkenstein_1 Fußgänger zum HZW': 'TFG Falkenstein 1',
'TFG_Falkenstein_2 Fußgänger In Richtung Parkplatz': 'TFG Falkenstein 2',
'TFG_Falkenstein_2 Fußgänger In Richtung TFG': 'TFG Falkenstein 2', 
'TFG_Lusen IN': 'TFG Lusen 1',
'TFG_Lusen OUT': 'TFG Lusen 1', 
'TFG_Lusen_1 Fußgänger Richtung TFG': 'TFG Lusen 1',
'TFG_Lusen_1 Fußgänger Richtung Parkplatz': 'TFG Lusen 1',
'TFG_Lusen_2 Fußgänger Richtung Vögel am Waldrand': 'TFG Lusen 2',
'TFG_Lusen_2 Fußgänger Richtung Parkplatz': 'TFG Lusen 2',
'TFG_Lusen_3 TFG Lusen 3 IN': 'TFG Lusen 3', 
'TFG_Lusen_3 TFG Lusen 3 OUT': 'TFG Lusen 3',
'Trinkwassertalsperre IN': 'Trinkwassertalsperre', 
'Trinkwassertalsperre OUT': 'Trinkwassertalsperre',
'Trinkwassertalsperre_MULTI IN': 'Trinkwassertalsperre', 
'Trinkwassertalsperre_MULTI OUT': 'Trinkwassertalsperre',
'Trinkwassertalsperre_MULTI Fußgänger IN': 'Trinkwassertalsperre',
'Trinkwassertalsperre_MULTI Fußgänger OUT': 'Trinkwassertalsperre',
'Trinkwassertalsperre_MULTI Fahrräder IN': 'Trinkwassertalsperre',
'Trinkwassertalsperre_MULTI Fahrräder OUT': 'Trinkwassertalsperre',
'Waldhausreibe IN': 'Waldhausreibe',
'Waldhausreibe OUT': 'Waldhausreibe', 
'Waldhausreibe Channel 1 IN': 'Waldhausreibe',
'Waldhausreibe Channel 2 OUT': 'Waldhausreibe', 
'Waldspielgelände_1 IN': 'Waldspielgelände', 
'Waldspielgelände_1 OUT': 'Waldspielgelände', 
'Wistlberg Fußgänger IN': 'Wistlberg',
'Wistlberg Fußgänger OUT': 'Wistlberg'}